In [1]:
import pandas as pd
import numpy as np

# from api_data import ApiData
from api_executer import ApiExecuter
from api_requester import ApiRequester
from api_parser import ApiParser
from FUNCTIONS import get_current_location

In [ ]:
location = get_current_location()
if location:
    latitude, longitude = location
    print(f'Current Latitude: {latitude}, Longitude: {longitude}')
else:
    print('Failed to determine current location.')

In [2]:
from api_route import ApiRoute
current_position = {'longitude': 130.844161,
                    'latitude': -12.456694}
final_position = {'longitude': 130.890346,
                  'latitude': -12.432393}
delta_spacing = 100.0 # in meters
number_sites = 150
route = ApiRoute()
# route.get_route_data(current_position)
# route.get_route_data(current_position, delta_spacing=delta_spacing)
dataframe = route.get_route_data(current_position=current_position, final_position=final_position, number_sites=number_sites)

      longitude   latitude   altitude  altitudeSmoothed  inclination  \
0    130.844161 -12.456694  18.000000         16.350559    -0.055933   
1    130.844452 -12.456401  15.455610         15.612559    -0.061814   
2    130.844738 -12.456102  12.644768         14.749027    -0.075971   
3    130.845013 -12.455794   9.738632         13.877752    -0.007226   
4    130.845268 -12.455470   9.415921         13.006129    -0.003268   
..          ...        ...        ...               ...          ...   
145  130.888672 -12.432395  16.714238         18.243455     0.016642   
146  130.889091 -12.432393  17.369049         18.771683     0.016474   
147  130.889509 -12.432388  18.230453         19.356345     0.013024   
148  130.889928 -12.432384  19.048181         19.946584     0.009826   
149  130.890346 -12.432393  19.500000         20.583545     0.008734   

     inclinationSmooth  inclinationSmoothed    distance  cumDistance  \
0            -0.016239            -0.024998   49.015820   407.0

In [ ]:
api_parser = ApiParser()
api_requester = ApiRequester(parser=api_parser)
api_executer = ApiExecuter(requester=api_requester)

In [ ]:
# api_executer.add_sites(dataframe, print_is_requested=False)
api_executer.delete_all_sites()

In [ ]:
position = {
    'latitude': -13.0,
    'longitude': 134.0
}
api_requester.get_site_edit(584978, True, name="aa")
api_requester.get_site_edit(584979, True, position=position)
api_requester.get_site_edit(584980, True, name="df", position=position)

In [ ]:
forecasts = api_requester.get_solar_forecast()
# forecasts = api_requester.get_solar_forecast_cloudmove()
print(forecasts)

In [ ]:
index = forecasts.index

# Access the values of time and site_id
time_values = index.get_level_values('time')
site_id_values = index.get_level_values('site_id')

# Print the values
print("Time values:", time_values)
print("Site ID values:", site_id_values)

In [ ]:
# Extract all 'rr' values for all site_id and time combinations
# rr_values = forecasts.loc[:, ('rr', slice(None), slice(None))].droplevel('variable')
rr_values = forecasts.loc[:, ('rh')]
rr_values = rr_values.to_frame()
print(rr_values)
print(rr_values.max(), rr_values.min())
# # Assuming your MultiIndex DataFrame is named "forecast_data"
# pivot_table = rr_values.pivot(index='time', columns='site_id', values='tt')
# print(pivot_table)
rr_values.to_csv('rh_table.csv')

# print(rr_values)

In [ ]:
import atmos
density = atmos.calculate('rho', Tv=300.0, p=990.0, p_units='hPa')
print(density)